<a href="https://colab.research.google.com/github/andersknudby/Remote-Sensing/blob/master/Chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 7 - Working with vector data
This chapter will help you learn how to read and write vector data, and how to work with vector data and image data in combination - specifically how to extract values from a raster for locations indeitifed in the vector file. Working with vector data in Python is in some ways similar to working with raster data – GDAL provides a core structure that includes a dataset with properties, and the dataset can include a number of layers, layers contain features, and features have attributes (like the ones you see in the attribute table for any shapefile when you open it in a GIS software). On top of GDAL, several other Python libraries have been created to make handling vector data easier and quicker. To start with we need to get rasterio again, and also a library called geopandas

In [ ]:
!pip install rasterio
!pip install geopandas

from shapely.geometry import Point
import rasterio
import geopandas as gpd

As usual, we also need to give Colab access to some files on our Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

myDir = '/content/drive/My Drive/Python files/'

import os
if os.path.exists(myDir + 'sfu.tif'):
  print("Drive mounted and directory found")
else:
  print("No access to the files")

Mounted at /content/drive
Drive mounted and directory found


Ok, that was the setup. On the Google Drive, in the Python files folder, there's a file called 'points.shp'. If you open it in QGIS you'll see that it's just six points, and if you overlay them on the sfu.tif image you'll see that they all fall inside that image, and that the two files have the same coordinate reference system, so overlaying them is straight-forward. but we can also very easily open the points.shp file here:

In [ ]:
pointsFilename = myDir + "points.shp"
pts = gpd.read_file(pointsFilename)  # I often use 'pts' as short for 'points' This is fairly common.
pts

,LandCover,Altitude,geometry
0,Forest,287,POINT (506722.856 5458623.040)
1,Forest,275,POINT (506902.335 5458647.466)
2,Forest,280,POINT (506706.926 5458652.776)
3,Parking lot,290,POINT (506693.651 5458566.223)
4,Parking lot,290,POINT (506707.988 5458563.037)
5,Parking lot,291,POINT (506728.166 5458552.417)


Easy, right? The 'pts' variable is an object called a **geodataframe**, which corresponds quite closely to the attribute table of the shapefile - each point is stored as a 'feature' in a row, and each attribute is stored in a column.

A **geodataframe** is based on a smaller kind of object from the pandas library, called a **dataframe**. But what makes a geodataframe special is that is always had geographic information associated with it, in the 'geometry' attribute. In the 'pts' data set you'll see that this information contains the label 'POINT', as well as the x and y coordinates of each point.

We can get some basic informatino about this geodataframe:

In [ ]:
print("Number of rows:", len(pts))
pts.crs

Number of rows: 7


<Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 126°W and 120°W, northern hemisphere between equator and 84°N, onshore and offshore. Canada - British Columbia (BC); Northwest Territories (NWT); Nunavut; Yukon. United States (USA) - Alaska (AK).
- bounds: (-126.0, 0.0, -120.0, 84.0)
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

You can work with geodataframes in may ways; here we will just look at some common ways to manipulate them.

If you want to remove a column, you can use the drop function:

In [ ]:
pts = pts.drop(['LandCover'], axis=1)
pts

,Altitude,geometry
0,287,POINT (506722.856 5458623.040)
1,275,POINT (506902.335 5458647.466)
2,280,POINT (506706.926 5458652.776)
3,290,POINT (506693.651 5458566.223)
4,290,POINT (506707.988 5458563.037)
5,291,POINT (506728.166 5458552.417)


And if you want to add a column you can do it like this:

In [ ]:
owner = ['SFU', 'SFU', 'SFU', 'SFU', 'SFU', 'SFU']
pts['Owner'] = owner
pts

,Altitude,geometry,Owner
0,287,POINT (506722.856 5458623.040),SFU
1,275,POINT (506902.335 5458647.466),SFU
2,280,POINT (506706.926 5458652.776),SFU
3,290,POINT (506693.651 5458566.223),SFU
4,290,POINT (506707.988 5458563.037),SFU
5,291,POINT (506728.166 5458552.417),SFU


Similarly, if you want to remove the first row:

In [ ]:
pts = pts.drop([0])
pts

,Altitude,geometry,Owner
1,275,POINT (506902.335 5458647.466),SFU
2,280,POINT (506706.926 5458652.776),SFU
3,290,POINT (506693.651 5458566.223),SFU
4,290,POINT (506707.988 5458563.037),SFU
5,291,POINT (506728.166 5458552.417),SFU


If you want to add a new row (or multiple rows) it gets a little more complicated, but not impossible:

In [ ]:
toAdd = [{'Altitude': 567, 'Owner': 'SFU', 'geometry': Point(506700, 5458600)},
         {'Altitude': 234, 'Owner': 'SFU', 'geometry': Point(506696, 5458612)}]

pts = pts.append(gpd.GeoDataFrame(toAdd))
pts

,Altitude,geometry,Owner
1,275,POINT (506902.335 5458647.466),SFU
2,280,POINT (506706.926 5458652.776),SFU
3,290,POINT (506693.651 5458566.223),SFU
4,290,POINT (506707.988 5458563.037),SFU
5,291,POINT (506728.166 5458552.417),SFU
0,567,POINT (506700.000 5458600.000),SFU
1,234,POINT (506696.000 5458612.000),SFU


## Extract values from a raster to a points shapefile
It is very common that you have a set up points, maybe location where you have been and observed something, and you want to extract values from a raster file at exactly those points. There are tools in ArcGIS and QGIS to do this, but it is also very easy to do with Python and geopandas. First we need to open the image file, as we did in the last chapter:

In [ ]:
imageFilename = myDir + 'sfu.tif'
ds = rasterio.open(imageFilename)  # ds is a commonly used shorthand for 'dataset'

Then we need to get all the (x,y) coordinates from the points shapefile. We can do that in a for loop, where we use i to iterate through the row numbers in pts. To do that we use the 'iloc' function, which returns the given row number from the geodataframe:

In [ ]:
coords = []
for i in range(len(pts)):
  x = pts.iloc[i].geometry.x
  y = pts.iloc[i].geometry.y
  coords.append((x, y))
coords

[(506902.3346581877, 5458647.466136725),
 (506706.926073132, 5458652.776152623),
 (506693.6510333864, 5458566.222893482),
 (506707.98807631165, 5458563.036883943),
 (506728.16613672505, 5458552.416852146),
 (506700.0, 5458600.0),
 (506696.0, 5458612.0)]

There are often other ways to do iterative things in Python than with a for loop. An alternative to the above is to use a list 'comprehension', which is more complex code to write, but requires fewer lines and executes faster. The result is exactly the same:

In [ ]:
coords = [(x, y) for x, y in zip(pts.geometry.x, pts.geometry.y)]
coords

[(506902.3346581877, 5458647.466136725),
 (506706.926073132, 5458652.776152623),
 (506693.6510333864, 5458566.222893482),
 (506707.98807631165, 5458563.036883943),
 (506728.16613672505, 5458552.416852146),
 (506700.0, 5458600.0),
 (506696.0, 5458612.0)]

When we haev the coordinates of all the points we can use the 'sample' function from rasterio to get the pixel values - in each band, for each of the point locations. For this we use another list comprehension. Remember that in our image the red bands was first, then green, and then blue. We need to feed that information as the 'indexes' argument to the sample function.

In [ ]:
pts["red"] = [x[0] for x in ds.sample(coords, indexes=1)]
pts["green"] = [x[0] for x in ds.sample(coords, indexes=2)]
pts["blue"] = [x[0] for x in ds.sample(coords, indexes=3)]
pts

,Altitude,geometry,Owner,red,green,blue
1,275,POINT (506902.335 5458647.466),SFU,46,66,54
2,280,POINT (506706.926 5458652.776),SFU,27,29,18
3,290,POINT (506693.651 5458566.223),SFU,147,166,172
4,290,POINT (506707.988 5458563.037),SFU,149,168,172
5,291,POINT (506728.166 5458552.417),SFU,141,160,166
0,567,POINT (506700.000 5458600.000),SFU,88,122,145
1,234,POINT (506696.000 5458612.000),SFU,41,51,39


Now, this works nicely because our two coordinate systems - for the raster and the vector data - line up. But what if they didn't? In that case you should first reproject the one data set to match the crs of the other. Typically the easiest, and certainly the fastest, is to reproject the vector data, because the number of calculations needed to reproject the raster data is much greater. Let's give it a try, even if we don't need to with these two data sets.

The first step is to actually test whether the two coordinate reference systems are the same. This could be done like this:

In [ ]:
epsgPoints = pts.crs.to_epsg()  # Get the EPSG number of the CRS of the vector data
epsgRaster = ds.crs.to_epsg()  # Get the EPSG number of the CRS of the vector data
epsgPoints == epsgRaster  # This tests whether the two EPSG numbers are the same.

True

But ok, let's imagine that the raster data instead used EPSG 32609 (i.e. one UTM zone to the west). Then we would need to reproject the points to use the coordinates from that UTM zone instead:

In [ ]:
newPts = pts.to_crs("EPSG:32609")

Now we'll try to repeat what we did above, but with the reprojected data. I know it's wrong, but it can be interesting see what happens:

In [ ]:
coords = [(x, y) for x, y in zip(newPts.geometry.x, newPts.geometry.y)]
pts["red"] = [x[0] for x in ds.sample(coords, indexes=1)]
pts["green"] = [x[0] for x in ds.sample(coords, indexes=2)]
pts["blue"] = [x[0] for x in ds.sample(coords, indexes=3)]
pts

,Altitude,geometry,Owner,red,green,blue
1,275,POINT (506902.335 5458647.466),SFU,NaN,NaN,NaN
2,280,POINT (506706.926 5458652.776),SFU,NaN,NaN,NaN
3,290,POINT (506693.651 5458566.223),SFU,NaN,NaN,NaN
4,290,POINT (506707.988 5458563.037),SFU,NaN,NaN,NaN
5,291,POINT (506728.166 5458552.417),SFU,NaN,NaN,NaN
0,567,POINT (506700.000 5458600.000),SFU,NaN,NaN,NaN
1,234,POINT (506696.000 5458612.000),SFU,NaN,NaN,NaN


This is another example of a semantic error, and a difficult one to find at that. Python thinks everything went well, and all the NaN values we now see in newPts indicate that the image has no values associated with the locations of the points. But that's because the 'coords' now refer to a different CRS than the coordinates in the image, and the 'sample' function has no way of knowing that.

**Important lesson:** It is up to you, when you write your code, to make sure that you check whether the coordinate reference systems of your data match! If they do not then Python may not show throw any errors, but your code may still be wrong!

The last thing we want to do is write the geodataframe back to a shapefile - given that we just did something wrong that is only going useful as an exercise:

In [ ]:
newPtsFilename = myDir + "newPoints.shp"
newPts.to_file(newPtsFilename)

And of course we need to unmount and flush to get the files on to Google Drive:

In [ ]:
drive.flush_and_unmount()

##Exercise
In a new notebook, write code that does the following:

1\) Read the 'sfu.tif' image.<br>
2\) Calculate the GCC of the image (see exercise from previous chapter).<br>
3\) Read the 'points.shp' file.<br>
4\) Extract the GCC value for each point.<br>
5\) Write the results as a new shapefile.<br>
6\) Open the original image and the new shapefile in QGIS, and check whether the points with the large GCC values are the ones located over vegetation.<br>


## Extract values from a raster to a polygon shapefile 
Sometimes it can be more meaningful to work with polygons than points, and we can also use Python to extract values from rasters to polygon shapefiles. Generically this is often called 'Zonal Statistics'. To do that we will install and use another library, called 'rasterstats':

In [ ]:
!pip install rasterstats
import rasterstats

     |████████████████████████████████| 133kB 5.9MB/s 


As the raster file we can work with the 'sfu.tif' image that is already loaded. And as the polygon data we can work with the file name 'polygons.shp':

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

myDir = '/content/drive/My Drive/Python files/'

import os
if os.path.exists(myDir + 'sfu.tif'):
  print("Drive mounted and directory found")
else:
  print("No access to the files")

polygonsFilename = myDir + "polygons.shp"
polys = gpd.read_file(polygonsFilename)  # I often use 'pts' as short for 'points' This is fairly common.
polys

Mounted at /content/drive
Drive mounted and directory found


,Type,geometry
0,Soccer field,"POLYGON ((506671.880 5458456.306, 506762.150 5..."
1,Roof,"POLYGON ((506827.994 5458444.093, 506869.413 5..."
2,Forest,"POLYGON ((506963.931 5458670.299, 506975.613 5..."


Let's try to find out which of the three polygons is the brightest. To do so we must first calculate the brightness of the image:

In [ ]:
import numpy as np  # We need NumPy for this

# And we need to open the image file again
imageFilename = myDir + 'sfu.tif'
ds = rasterio.open(imageFilename)  # ds is a commonly used shorthand for 'dataset'

band1 = ds.read(1).astype('uint16')
band2 = ds.read(2).astype('uint16')
band3 = ds.read(3).astype('uint16')
brightness = (band1 + band2 + band3) / 3
brightness

array([[ 43.33333333,  49.33333333,  54.        , ...,  57.66666667,
         58.66666667,  56.33333333],
       [ 34.66666667,  35.66666667,  47.        , ...,  57.33333333,
         53.66666667,  52.        ],
       [ 41.66666667,  38.66666667,  40.33333333, ...,  45.        ,
         49.        ,  53.        ],
       ...,
       [ 88.66666667,  90.        , 105.        , ...,  28.        ,
         26.33333333,  25.        ],
       [ 70.66666667,  76.        ,  99.        , ...,  26.33333333,
         24.33333333,  24.        ],
       [ 57.66666667,  70.33333333, 100.        , ...,  25.        ,
         23.66666667,  24.33333333]])

Now brightness is a NumPy array, but there are two remaining issues:

1\) NumPy arrays don't have location information attached to them, so we need to extract that information from the raster data as well. Otherwise there is no way to associated the coordinates for each polygon with the rows and columns of the NumPy array! To do that we can use the geotransform information from ds, using 'ds.transform'

2\) We also need to provide information on the value, in the NumPy array, that is used for missing data. This is the standard 'nan' value used by NumPy.

So, in the end we can use the rasterstats zonal_stats function like this:


In [ ]:
stats = rasterstats.zonal_stats(polys, band1, nodata=np.nan, affine=ds.transform)  # This works
stats

[{'count': 480124, 'max': 251.0, 'mean': 140.33809599186876, 'min': 42.0},
 {'count': 62428, 'max': 237.0, 'mean': 74.28887678605754, 'min': 9.0},
 {'count': 93533, 'max': 100.0, 'mean': 35.84866303871361, 'min': 7.0}]

Note that the zonal_stats function can also work with files directly, instead of with data objects already read into Python. Here are two examples that would provide the same results as the above:

In [ ]:
stats = rasterstats.zonal_stats(polygonsFilename, imageFilename)  # Using files for both vector and raster data
stats = rasterstats.zonal_stats(polys, imageFilename)  # Using the 'polys' object with the raster file

##Exercise
Find out which of the three polygons has the highest mean GCC value.